# 학습 패턴 분석

TimeLevelUp 학생들의 학습 패턴을 분석합니다.

## 분석 항목
1. 요일별 학습량 분포
2. 시간대별 학습 패턴
3. 과목별 학습 시간 분석
4. 완료율 및 이행률 분석

In [ ]:
# 의존성 설치 (첫 실행 시)
# !pip install pandas numpy matplotlib seaborn supabase python-dotenv

In [ ]:
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# 한글 폰트 설정
plt.rcParams['font.family'] = 'AppleGothic'  # macOS
# plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows
plt.rcParams['axes.unicode_minus'] = False

# 스타일 설정
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
from db_connector import get_connector
from analysis import analyze_learning_patterns, predict_weekly_workload

# Supabase 연결
db = get_connector()
print('Supabase 연결 성공!')

## 1. 데이터 로드

In [ ]:
# 분석할 학생 ID 설정 (실제 ID로 변경 필요)
STUDENT_ID = 'YOUR_STUDENT_ID_HERE'

# 최근 3개월 데이터 조회
three_months_ago = (datetime.now() - timedelta(days=90)).strftime('%Y-%m-%d')

# 학습 플랜 데이터
plans_df = db.get_student_plans(STUDENT_ID, start_date=three_months_ago)
print(f'학습 플랜 수: {len(plans_df)}')
plans_df.head()

## 2. 요일별 학습량 분석

In [ ]:
if not plans_df.empty and 'scheduled_date' in plans_df.columns:
    plans_df['date'] = pd.to_datetime(plans_df['scheduled_date'])
    plans_df['day_of_week'] = plans_df['date'].dt.dayofweek
    plans_df['day_name'] = plans_df['date'].dt.day_name()
    
    day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    day_names_kr = ['월', '화', '수', '목', '금', '토', '일']
    
    daily_counts = plans_df.groupby('day_of_week').size()
    
    fig, ax = plt.subplots(figsize=(10, 6))
    bars = ax.bar(day_names_kr, [daily_counts.get(i, 0) for i in range(7)], color='steelblue')
    ax.set_xlabel('요일')
    ax.set_ylabel('플랜 수')
    ax.set_title('요일별 학습 플랜 분포')
    
    # 평균선 표시
    avg = daily_counts.mean()
    ax.axhline(y=avg, color='red', linestyle='--', label=f'평균: {avg:.1f}')
    ax.legend()
    
    plt.tight_layout()
    plt.show()
else:
    print('데이터가 없거나 scheduled_date 컬럼이 없습니다.')

## 3. 시간대별 학습 패턴

In [ ]:
if not plans_df.empty and 'start_time' in plans_df.columns:
    plans_df['hour'] = pd.to_datetime(plans_df['start_time'], format='%H:%M', errors='coerce').dt.hour
    hourly_counts = plans_df.dropna(subset=['hour']).groupby('hour').size()
    
    fig, ax = plt.subplots(figsize=(14, 6))
    hours = range(24)
    counts = [hourly_counts.get(h, 0) for h in hours]
    
    bars = ax.bar(hours, counts, color='teal', alpha=0.7)
    ax.set_xlabel('시간')
    ax.set_ylabel('플랜 수')
    ax.set_title('시간대별 학습 플랜 분포')
    ax.set_xticks(hours)
    ax.set_xticklabels([f'{h}시' for h in hours], rotation=45)
    
    # 피크 시간 강조
    if hourly_counts.any():
        peak_hour = hourly_counts.idxmax()
        bars[peak_hour].set_color('coral')
        ax.annotate(f'피크: {peak_hour}시', xy=(peak_hour, counts[peak_hour]), 
                    xytext=(peak_hour+1, counts[peak_hour]+2), fontsize=10)
    
    plt.tight_layout()
    plt.show()
else:
    print('start_time 데이터가 없습니다.')

## 4. 과목별 학습 시간 분석

In [ ]:
if not plans_df.empty and 'subject' in plans_df.columns:
    subject_counts = plans_df['subject'].value_counts()
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    # 막대 그래프
    subject_counts.plot(kind='bar', ax=ax1, color='steelblue')
    ax1.set_xlabel('과목')
    ax1.set_ylabel('플랜 수')
    ax1.set_title('과목별 학습 플랜 수')
    ax1.tick_params(axis='x', rotation=45)
    
    # 파이 차트
    ax2.pie(subject_counts.values, labels=subject_counts.index, autopct='%1.1f%%', startangle=90)
    ax2.set_title('과목별 학습 비율')
    
    plt.tight_layout()
    plt.show()
else:
    print('subject 데이터가 없습니다.')

## 5. 완료율 분석

In [ ]:
if not plans_df.empty and 'status' in plans_df.columns:
    status_counts = plans_df['status'].value_counts()
    
    colors = {
        'completed': 'green',
        'pending': 'orange',
        'in_progress': 'blue',
        'skipped': 'red',
    }
    
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.pie(status_counts.values, 
           labels=status_counts.index, 
           autopct='%1.1f%%',
           colors=[colors.get(s, 'gray') for s in status_counts.index],
           startangle=90,
           explode=[0.05 if s == 'completed' else 0 for s in status_counts.index])
    ax.set_title('학습 플랜 상태 분포')
    
    # 완료율 표시
    completed = status_counts.get('completed', 0)
    total = status_counts.sum()
    completion_rate = completed / total * 100 if total > 0 else 0
    ax.text(0, -1.3, f'완료율: {completion_rate:.1f}%', ha='center', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
else:
    print('status 데이터가 없습니다.')

## 6. 종합 분석 리포트

In [ ]:
# 분석 함수 실행
analysis_result = analyze_learning_patterns(plans_df)

print('=' * 50)
print('학습 패턴 분석 리포트')
print('=' * 50)

for key, value in analysis_result.items():
    print(f'\n### {key.replace("_", " ").title()}')
    if isinstance(value, dict):
        for k, v in value.items():
            print(f'  - {k}: {v}')
    else:
        print(f'  {value}')

## 7. 다음 주 학습량 예측

In [ ]:
prediction = predict_weekly_workload(plans_df)

print('\n다음 주 학습량 예측')
print('-' * 30)
for key, value in prediction.items():
    print(f'{key}: {value}')